<a href="https://colab.research.google.com/github/ouda711/Machine-Learning/blob/main/Data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [175]:
import io

import pandas as pd
import numpy as np
from google.colab import files
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.metrics import accuracy_score

In [2]:
upload = files.upload()['Housing Pricing.csv']

Saving Housing Pricing.csv to Housing Pricing (1).csv


In [12]:
data = io.BytesIO(upload)
data_frame = pd.read_csv(data)
data_frame.head(10)
data_frame.drop('Id',inplace=True, axis=1)

In [4]:
# Find features with missing values
features_missing_values = list(data_frame.columns[data_frame.isna().any()])
features_missing_values

['LotFrontage',
 'MasVnrType',
 'MasVnrArea',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageQual',
 'GarageCond']

In [5]:
mean_fill, back_fill = [], []
for feat in features_missing_values:
  if data_frame[feat].dtype == "float64":
    mean_fill.append(feat)
  else:
    back_fill.append(feat)

In [6]:
for feat in mean_fill:
  data_frame.fillna(data_frame[feat].mean(),inplace=True)

In [7]:
for feat in back_fill:
  data_frame.fillna(method='bfill',inplace=True)

In [8]:
# Check for remaining features with missing values

sum(data_frame.columns[data_frame.isna().any()])

0

In [9]:
non_numerical_features = list(data_frame.select_dtypes(include='object').columns)
numerical_features = list(data_frame.select_dtypes(exclude='object').columns)
print(numerical_features)
print(non_numerical_features)

['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice']
['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'Garag

In [18]:
data_frame[non_numerical_features].head(100).tail(20)

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
80,3,1,3,3,0,0,0,12,2,2,0,5,1,1,12,13,BrkFace,TA,Gd,CBlock,Gd,TA,No,Rec,Unf,GasA,TA,Y,SBrkr,Gd,Typ,Gd,Attchd,Fin,TA,TA,Y,WD,Normal
81,4,1,3,3,0,2,0,11,2,2,4,2,3,1,12,13,BrkFace,TA,Gd,PConc,Ex,Gd,No,GLQ,Unf,GasA,Ex,Y,SBrkr,TA,Typ,NaN,Attchd,Fin,TA,TA,Y,WD,Normal
82,3,1,3,3,0,4,0,21,2,2,0,2,1,1,12,13,Stone,TA,TA,PConc,Gd,TA,No,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,Gd,Attchd,RFn,TA,TA,Y,WD,Normal
83,3,1,0,3,0,4,0,12,2,2,0,2,1,1,8,8,BrkCmn,TA,TA,CBlock,TA,TA,No,Unf,Unf,GasA,Gd,Y,SBrkr,TA,Typ,NaN,Detchd,Unf,TA,TA,Y,COD,Normal
84,3,1,0,3,0,4,0,8,2,2,0,7,1,1,6,6,BrkFace,TA,TA,PConc,Gd,TA,No,Unf,Unf,GasA,Gd,Y,SBrkr,TA,Typ,TA,BuiltIn,Fin,TA,TA,Y,WD,Normal
85,3,1,3,3,0,0,0,15,2,2,0,5,3,1,6,6,BrkFace,Gd,TA,CBlock,Gd,TA,No,Unf,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,TA,Attchd,Unf,TA,TA,Y,WD,Normal
86,3,1,1,3,0,4,0,8,2,2,0,5,1,1,12,13,None,Gd,TA,PConc,Gd,TA,Av,Unf,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,Gd,BuiltIn,Fin,TA,TA,Y,WD,Normal
87,1,1,3,3,0,0,0,21,2,2,4,5,1,1,12,13,Stone,Gd,TA,PConc,Gd,TA,Av,Unf,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,NaN,Detchd,RFn,TA,TA,Y,New,Partial
88,0,1,0,3,0,0,0,9,1,1,0,0,3,1,9,10,None,Fa,Fa,CBlock,TA,Fa,No,Unf,Unf,GasA,TA,N,SBrkr,Fa,Typ,NaN,NaN,NaN,NaN,NaN,N,ConLD,Abnorml
89,3,1,3,3,0,4,0,5,2,2,0,2,1,1,12,13,None,TA,TA,PConc,Gd,TA,No,GLQ,Unf,GasA,Ex,Y,SBrkr,TA,Typ,NaN,NaN,NaN,NaN,NaN,Y,WD,Normal


In [20]:
nominal = ['MSZoning','Street','RoofStyle','Neighborhood']
ordinal = list(set(non_numerical_features) - set(nominal))

In [21]:
ordinal

['BsmtFinType1',
 'KitchenQual',
 'Exterior2nd',
 'SaleType',
 'LotShape',
 'GarageType',
 'GarageFinish',
 'LandContour',
 'Condition2',
 'Exterior1st',
 'Utilities',
 'RoofMatl',
 'Heating',
 'BsmtExposure',
 'BldgType',
 'Functional',
 'BsmtFinType2',
 'MasVnrType',
 'FireplaceQu',
 'Condition1',
 'HouseStyle',
 'SaleCondition',
 'Foundation',
 'ExterCond',
 'BsmtQual',
 'CentralAir',
 'ExterQual',
 'GarageQual',
 'LotConfig',
 'BsmtCond',
 'Electrical',
 'PavedDrive',
 'LandSlope',
 'GarageCond',
 'HeatingQC']

In [22]:
le = LabelEncoder()

In [42]:
# data_frame.MSZoning = le.fit_transform(data_frame.MSZoning)
# data_frame.Street = le.fit_transform(data_frame.Street)
# data_frame.LotShape = le.fit_transform(data_frame.LotShape)
# data_frame.LandContour = le.fit_transform(data_frame.LandContour)
# data_frame.Utilities = le.fit_transform(data_frame.Utilities)
# data_frame.LotConfig = le.fit_transform(data_frame.LotConfig)
# data_frame.LandSlope = le.fit_transform(data_frame.LandSlope)
# data_frame.Neighborhood = le.fit_transform(data_frame.Neighborhood)
# data_frame.Condition1 = le.fit_transform(data_frame.Condition1)
# data_frame.Condition2 = le.fit_transform(data_frame.Condition2)
# data_frame.BldgType = le.fit_transform(data_frame.BldgType)
# data_frame.HouseStyle = le.fit_transform(data_frame.HouseStyle)
# data_frame.RoofMatl = le.fit_transform(data_frame.RoofMatl)
# data_frame.RoofStyle = le.fit_transform(data_frame.RoofStyle)
# data_frame.Exterior1st = le.fit_transform(data_frame.Exterior1st)
# data_frame.Exterior2nd = le.fit_transform(data_frame.Exterior2nd)
# data_frame.MasVnrType = le.fit_transform(data_frame.MasVnrType.astype(str))
# data_frame.ExterCond = le.fit_transform(data_frame.ExterCond)
# data_frame.ExterQual = le.fit_transform(data_frame.ExterQual)
# data_frame.Foundation = le.fit_transform(data_frame.Foundation)
# data_frame.BsmtCond = le.fit_transform(data_frame.BsmtCond.astype(str))
# data_frame.BsmtCond = le.fit_transform(data_frame.BsmtCond)
# data_frame.BsmtQual = le.fit_transform(data_frame.BsmtQual.astype(str))
# data_frame.BsmtExposure = le.fit_transform(data_frame.BsmtExposure.astype(str))
# data_frame.BsmtFinType1 = le.fit_transform(data_frame.BsmtFinType1.astype(str))
# data_frame.BsmtFinType2 = le.fit_transform(data_frame.BsmtFinType2.astype(str))
# data_frame.Heating = le.fit_transform(data_frame.Heating)
# data_frame.HeatingQC = le.fit_transform(data_frame.HeatingQC)
# data_frame.CentralAir = le.fit_transform(data_frame.CentralAir)
# data_frame.Electrical = le.fit_transform(data_frame.Electrical.astype(str))
# data_frame.KitchenQual = le.fit_transform(data_frame.KitchenQual)
# data_frame.Functional = le.fit_transform(data_frame.Functional)
# data_frame.FireplaceQu = le.fit_transform(data_frame.FireplaceQu.astype(str))
# data_frame.GarageType = le.fit_transform(data_frame.GarageType.astype(str))
# data_frame.GarageFinish = le.fit_transform(data_frame.GarageFinish.astype(str))
# data_frame.GarageArea = le.fit_transform(data_frame.GarageArea)
# data_frame.GarageQual = le.fit_transform(data_frame.GarageQual.astype(str))
# data_frame.GarageCond = le.fit_transform(data_frame.GarageCond.astype(str))
# data_frame.PavedDrive = le.fit_transform(data_frame.PavedDrive)
# data_frame.SaleType = le.fit_transform(data_frame.SaleType)
# data_frame.SaleCondition = le.fit_transform(data_frame.SaleCondition)

In [60]:
categorical_columns = data_frame.columns[data_frame.dtypes == object]
df = pd.get_dummies(data_frame, columns = categorical_columns, drop_first=True)
df

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,3,65.0,8450,1,3,3,0,4,0,5,2,2,0,5,7,5,2003,2003,1,1,12,13,1,196.0,2,4,2,2,3,3,2,706,5,0,150,856,1,0,1,4,856,854,0,1710,1,0,2,1,3,1,2,8,6,0,5,1,2003.0,1,2,220,4,4,2,0,61,0,0,0,0,0,2,2008,8,4,208500
1,20,3,80.0,9600,1,3,3,0,2,0,24,1,2,0,2,6,8,1976,1976,1,1,8,8,2,0.0,3,4,1,2,3,1,0,978,5,0,284,1262,1,0,1,4,1262,0,0,1262,0,1,2,0,3,1,3,6,6,1,4,1,1976.0,1,2,149,4,4,2,298,0,0,0,0,0,0,5,2007,8,4,181500
2,60,3,68.0,11250,1,0,3,0,4,0,5,2,2,0,5,7,5,2001,2002,1,1,12,13,1,162.0,2,4,2,2,3,2,2,486,5,0,434,920,1,0,1,4,920,866,0,1786,1,0,2,1,3,1,2,6,6,1,4,1,2001.0,1,2,260,4,4,2,0,42,0,0,0,0,0,9,2008,8,4,223500
3,70,3,60.0,9550,1,0,3,0,0,0,6,2,2,0,5,7,5,1915,1970,1,1,13,15,2,0.0,3,4,0,3,1,3,0,216,5,0,540,756,1,2,1,4,961,756,0,1717,1,0,1,0,3,1,2,7,6,1,2,5,1998.0,2,3,284,4,4,2,0,35,272,0,0,0,0,2,2006,8,0,140000
4,60,3,84.0,14260,1,0,3,0,2,0,15,2,2,0,5,8,5,2000,2000,1,1,12,13,1,350.0,2,4,2,2,3,0,2,655,5,0,490,1145,1,0,1,4,1145,1053,0,2198,1,0,2,1,4,1,2,9,6,1,4,1,2000.0,1,3,380,4,4,2,192,84,0,0,0,0,0,12,2008,8,4,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,3,62.0,7917,1,3,3,0,4,0,8,2,2,0,5,6,5,1999,2000,1,1,12,13,2,0.0,3,4,2,2,3,3,5,0,5,0,953,953,1,0,1,4,953,694,0,1647,0,0,2,1,3,1,3,7,6,1,4,1,1999.0,1,2,149,4,4,2,0,40,0,0,0,0,0,8,2007,8,4,175000
1456,20,3,85.0,13175,1,3,3,0,4,0,14,2,2,0,2,6,6,1978,1988,1,1,9,10,3,119.0,3,4,1,2,3,3,0,790,4,163,589,1542,1,4,1,4,2073,0,0,2073,1,0,2,0,3,1,3,7,2,2,4,1,1978.0,2,2,182,4,4,2,349,0,0,0,0,0,0,2,2010,8,4,210000
1457,70,3,66.0,9042,1,3,3,0,4,0,6,2,2,0,5,7,9,1941,2006,1,1,5,5,2,0.0,0,2,4,3,1,3,2,275,5,0,877,1152,1,0,1,4,1188,1152,0,2340,0,0,2,0,4,1,2,9,6,2,2,1,1941.0,1,1,24,4,4,2,0,60,0,0,0,0,2500,5,2010,8,4,266500
1458,20,3,68.0,9717,1,3,3,0,4,0,12,2,2,0,2,5,6,1950,1996,3,1,8,8,2,0.0,3,4,1,3,3,2,2,49,4,1029,0,1078,1,2,1,0,1078,0,0,1078,1,0,1,0,2,1,2,5,6,0,5,1,1950.0,2,1,19,4,4,2,366,0,112,0,0,0,0,4,2010,8,4,142125


In [125]:
numerics = ['int16','int32','int64','float16','float32','float64']
numerical_vars = list(data_frame.select_dtypes(include=numerics).columns)
df = df[numerical_vars]
df.shape

(1460, 76)

In [126]:
X_train, X_test, y_train, y_test = X_train, X_test, y_train, y_test = train_test_split(df[df.columns[df.columns != 'SalePrice']],
                   df['SalePrice'], test_size=0.25, random_state=1)
print(X_train, X_test, y_train, y_test)

      MSSubClass  MSZoning  LotFrontage  ...  YrSold  SaleType  SaleCondition
6             20         3         75.0  ...    2007         8              4
807           70         3        144.0  ...    2009         8              4
955           90         2         82.0  ...    2007         8              4
1040          20         3         88.0  ...    2006         8              4
701           20         3         80.0  ...    2006         0              4
...          ...       ...          ...  ...     ...       ...            ...
715           20         3         78.0  ...    2009         8              4
905           20         3         80.0  ...    2010         8              4
1096          70         4         60.0  ...    2007         8              4
235          160         4         21.0  ...    2008         8              4
1061          30         0        120.0  ...    2008         3              4

[1095 rows x 75 columns]       MSSubClass  MSZoning  LotFrontag

In [110]:
sc = StandardScaler()
sc.fit(X_train.fillna(0))
# X_train=X_train.fillna(X_train.mean())
# X_test=X_test.fillna(X_test.mean())
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [111]:
pca = PCA()
#
# Determine transformed features
#
X_train_pca = pca.fit_transform(X_train_std)
X_test_pca = pca.transform(X_test_std)

In [132]:
sel_ = SelectFromModel(LogisticRegression(C=1, penalty='l1', solver='liblinear'))
sel_.fit(sc.transform(X_train.fillna(0)), y_train)

SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='auto',
                                             n_jobs=None, penalty='l1',
                                             random_state=None,
                                             solver='liblinear', tol=0.0001,
                                             verbose=0, warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [133]:
sel_.get_support()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [134]:
selected_feat = X_train.columns[(sel_.get_support())]
print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('features with coefficients shrank to zero: {}'.format(
      np.sum(sel_.estimator_.coef_ == 0)))

total features: 75
selected features: 75
features with coefficients shrank to zero: 33661


In [135]:
np.sum(sel_.estimator_.coef_ == 0)

33661

In [137]:
X_train_selected = sel_.transform(X_train.fillna(0))
X_test_selected = sel_.transform(X_test.fillna(0))
X_train_selected.shape, X_test_selected.shape

((1095, 75), (365, 75))

In [141]:
pca = PCA(.95)
pca.fit(X_train_selected)

PCA(copy=True, iterated_power='auto', n_components=0.95, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [142]:
train_data = pca.transform(X_train_selected)
test_data = pca.transform(X_test_selected)

In [146]:
regr = LinearRegression()
regr.fit(train_data[:,:1], y_train)

#calculate RMSE
pred = regr.predict(test_data)
np.sqrt(mean_squared_error(y_test, pred))

79659.76290959182

In [147]:
print(pca.components_)

[[-7.39865074e-04 -2.43612625e-06  4.29379186e-04  9.99676799e-01
  -5.69642475e-08 -2.22514573e-05 -9.93938265e-06  3.32833523e-08
  -1.81355575e-05  1.20701593e-05  4.50603298e-06  2.15669056e-06
   6.28527458e-07 -2.55979076e-05 -4.10309794e-06  1.53537840e-05
  -7.64803967e-07  2.58746370e-05  3.96985093e-05  6.87681297e-06
   8.44263115e-06 -1.17059578e-06 -6.81446576e-06 -6.96599174e-07
   2.12385838e-03 -4.21953703e-06  7.49696130e-07 -1.05266124e-06
  -6.60079979e-06 -3.11941460e-07 -1.68044065e-05 -1.02155876e-05
   1.00218114e-02 -3.24572734e-06  2.10784232e-03 -8.47245067e-05
   1.20449292e-02  1.08761001e-06  5.05426206e-06  1.28058205e-06
   5.42384202e-06  1.22556549e-02  2.38436764e-03  6.37862663e-05
   1.47038088e-02  7.03087174e-06  1.58628904e-06  7.29049100e-06
   5.65219802e-07  1.10233052e-05 -4.11861265e-07 -4.79670543e-06
   3.33247828e-05 -4.12746289e-06  1.79825584e-05 -2.03770920e-05
  -2.67781730e-05  3.23101391e-03 -9.80124968e-06  1.14257776e-05
   1.90340

In [148]:
print(pca.explained_variance_)

[1.0345419e+08]


In [160]:
print("original shape:   ", X_train.fillna(0).shape)
print("transformed shape:", train_data.shape)

original shape:    (1095, 75)
transformed shape: (1095, 1)
